In [ ]:
import os
import shutil
import json 
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon
from shapely.geometry import Point

In [ ]:
directory = "data/tables/pagexml"
image_name = "NL-HaNA_2.10.50_45_0110.jpg"

need to construct HTML Table form transkribus XML

In [ ]:
from src.utils import pagexml_to_html

pagexml_file = os.path.join(directory, image_name+ ".xml")
output_file = os.path.join("data/tables/html", image_name + ".html")
pagexml_to_html(pagexml_file, output_file)

### Measure mAP

In [ ]:
from src.metrics import compute_mAP

gt_file = "data/labels/polygons/NL-HaNA_2.10.50_45_0091.jpg.polygons.json"
pred_file = "data/tables/pagexml/NL-HaNA_2.10.50_45_0091.jpg.xml"
mAP = compute_mAP(gt_file, pred_file)

### Measure TED

In [ ]:
from src.utils import format_td
from src.metrics import TEDS
def calculate_TEDS(ground_truth_html, predicted_html):
    # predicted_html = format_td(predicted_html)
    ground_truth_html = format_td(ground_truth_html)

    teds = TEDS(structure_only=False)
    teds_score = teds.evaluate(ground_truth_html, predicted_html)

    teds_struct = TEDS(structure_only=True)
    teds_struct_score = teds_struct.evaluate(ground_truth_html, predicted_html)
    
    print(f"TEDS: {teds_score:.4f}")
    print(f"TEDS-Struct: {teds_struct_score:.4f}")

    return teds_score, teds_struct_score

In [ ]:
import re
with open(os.path.join("data/tables/html/", image_name+ '.html'), 'r', encoding='utf-8') as f:
    constructed_html = f.read()

with open(os.path.join("data/labels/tables", image_name+ '.html'), 'r', encoding='utf-8') as f:
    label_html = f.read()

calculate_TEDS(label_html, constructed_html)

### Information Extraction (using ONTOGPT)

In [ ]:
import os
import shutil
import json 

image_name = "NL-HaNA_2.10.50_45_0091.jpg"
with open(f"data/tables/html{image_name}.html", 'r', encoding='utf-8') as f:
    constructed_html = f.read()

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(constructed_html, 'html.parser')

rows = soup.find_all('tr')

# --- Step 1: make a grid keeping track of rowspans ---
logical_rows = []
rowspans = {}  # {(col_idx): [remaining_rows, cell_content]}

for r_idx, tr in enumerate(rows):
    # Start with any carried over rowspans from previous rows
    current_row = []
    to_remove = []

    # Fill carried-down cells first
    for col_idx, (remaining, cell) in rowspans.items():
        current_row.append(cell)
        rowspans[col_idx][0] -= 1
        if rowspans[col_idx][0] <= 0:
            to_remove.append(col_idx)
    for col_idx in to_remove:
        del rowspans[col_idx]

    # Now add new cells from this row
    c_idx = 0
    for td in tr.find_all('td'):
        while any(k == c_idx for k in rowspans):  # skip columns occupied by carried cells
            c_idx += 1
        text = td.get_text(" ", strip=True)
        cell_id = td.get('id')
        r_idx = int(td.get('row', r_idx))
        c_idx = int(td.get('col', c_idx))
        rowspan = int(td.get('rowspan', 1))
        colspan = int(td.get('colspan', 1))

        cell_data = {
            'text': text,
            'id': cell_id,
            'row': r_idx,
            'col': c_idx,
            'rowspan': rowspan,
            'colspan': colspan
        }

        # Place cell in current row
        current_row.append(cell_data)

        # Store for future rows if rowspan > 1
        if rowspan > 1:
            rowspans[c_idx] = [rowspan - 1, cell_data]

        c_idx += colspan

    logical_rows.append(current_row)


In [3]:
from src.person_info_extraction_ontogpt import extract_person_info as extractor
 

temp_dir = "data/temp/"
os.makedirs(temp_dir, exist_ok=True)

for i, row in enumerate(logical_rows):
    print(f"Processing row {i+1}/{len(logical_rows)}")
    extracted_info = extractor(row, schema_path="data/schema/personbasicinfo.yaml", json_output=f"data/temp/person_{i}.json",temp_dir=temp_dir, llm_model="ollama/llama3")

# concate json objects from differnt files and same it in a single file

persons = []

for filename in os.listdir("data/temp/"):
    if filename.endswith(".json") and filename.startswith("person_"):
        with open(os.path.join("data/temp/", filename), 'r', encoding='utf-8') as f:
            data = json.load(f)
            # each temp file is expected to be {'persons': [...]}
            person = data.get("persons", [])
            if isinstance(persons, list):
                persons.extend(person)
            elif person:
                persons.append(persons)

with open(f"data/json/{image_name}.json", 'w', encoding='utf-8') as f:
    json.dump({"persons": persons}, f, indent=2, ensure_ascii=False)    

# Step 6: Cleanup
shutil.rmtree(temp_dir, ignore_errors=True)


Processing row 1/3
Running OntoGPT...

Processing row 2/3
Running OntoGPT...

Processing row 3/3
Running OntoGPT...



# Generate Information for ontoGPT

In [ ]:
import yaml

cell_spans = []     # list of dicts: {id, start, end}
row_text = ""       # concatenated text of the entire table
cursor = 0          # tracks current char offset

for i, row in enumerate(logical_rows):
    if i != 1:
        continue
    for cell in row:
        text = cell["text"]
        cid = cell["id"]

        start = cursor
        end = start + len(text)

        cell_spans.append({
            "id": cid,
            "start": start,
            "end": end,
            "text": text
        })

        row_text += text + '\n'
        cursor = end

    with open("table_cells.yaml", "w", encoding="utf-8") as f:
        yaml.dump(cell_spans, f, allow_unicode=True, sort_keys=False)

    with open("row.txt", "w+") as f:
        f.write(row_text)

In [ ]:
%%bash
ontogpt extract -i row.txt -t personbasicinfo.yaml -m ollama/llama3 -o person.yaml


### map original_spans to cell

In [ ]:
import yaml

def find_cell_for_span(start, spans):
    """
    Return the cell_id whose text covers the character span starting at 'start'.
    Span belongs to the cell where:  cell.start <= start < cell.end
    """
    for item in spans:
        if item["start"] <= start < item["end"]:
            return item["id"]
    return None


# ------------------------------
# 1. Load ONTOGPT output file
# ------------------------------

yaml_path = "person.yaml"     
with open(yaml_path, "r", encoding="utf-8") as f:
    data = yaml.safe_load(f)

named_entities = data.get("named_entities", [])

# ------------------------------
# 2. Load cell_spans index
# ------------------------------

yaml_path = "table_cells.yaml"     
with open(yaml_path, "r", encoding="utf-8") as f:
    cell_spans = yaml.safe_load(f)


# ------------------------------
# 3. For each named entity, map span to cell id
# ------------------------------

for ent in named_entities:
    spans = ent.get("original_spans", [])
    ent_cells = []

    for span_str in spans:
        try:
            start, end = map(int, span_str.split(":"))
        except:
            continue

        cell_id = find_cell_for_span(start, cell_spans)
        ent_cells.append(cell_id)

    # Add new slot "cell"
    if ent_cells:
        # if only one span, store a single value
        ent["cell"] = ent_cells[0] if len(ent_cells) == 1 else ent_cells
    else:
        ent["cell"] = None



# ------------------------------
# 4. Save updated YAML
# ------------------------------

output_path = "person.yaml"
with open(output_path, "w", encoding="utf-8") as f:
    yaml.dump(data, f, sort_keys=False, allow_unicode=True)

print("✓ Updated YAML saved to", output_path)


### Convert YAML to json

In [ ]:
import yaml
import json
from copy import deepcopy


def load_yaml(path):
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)


def get_entity_for_value(raw_value, named_entities):
    """
    Given a raw value like 'AUTO:Wageningen',
    return the full named_entity dict (label, cell, original_spans),
    or None if not found.
    """
    if not isinstance(raw_value, str):
        return None

    for ent in named_entities:
        if ent.get("id") == raw_value:
            return ent

    return None


def process_field(value, named_entities):
    """
    Converts a YAML value into normalized JSON format with:
    - value
    - cell
    - original_spans
    Handles nested dicts.
    """

    # Case 1: nested dict → process each subfield recursively
    if isinstance(value, dict):
        processed = {}
        for k, v in value.items():
            processed[k] = process_field(v, named_entities)
        return processed

    # Case 2: literal field or AUTO reference
    ent = get_entity_for_value(value, named_entities)

    if ent:
        # match found: use curated label + provenance
        return {
            "value": ent.get("label"),
            "cell": ent.get("cell"),
            "original_spans": ent.get("original_spans")
        }

    # No entity match → raw literal value
    return {
        "value": value,
        "cell": None,
        "original_spans": None
    }


def convert_yaml_to_person_json(data):
    extracted = deepcopy(data["extracted_object"])
    named_entities = data["named_entities"]

    person = {}

    # Process all fields dynamically
    for key, value in extracted.items():
        person[key] = process_field(value, named_entities)

    return {"persons": [person]}


def write_json(path, obj):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, ensure_ascii=False)


# -------------------------------
# Example usage
# -------------------------------
if __name__ == "__main__":
    yaml_path = "person.yaml"
    json_out = "person.json"

    data = load_yaml(yaml_path)
    result = convert_yaml_to_person_json(data)
    write_json(json_out, result)

    print(json.dumps(result, indent=2, ensure_ascii=False))


# Information Extraction Module

In [ ]:
import re
import json
from src.person_info_extraction import extract_info_regex, extract_info_LLM

persons = []

for i, row in enumerate(logical_rows):
    person = {}
        
    # print(generate_prompt(cells))
    person = json.loads(extract_info_LLM(row))
    if all(v['value']==None for v in person.values()):
        continue

    if person:
        persons.append(person)

unique_persons = {json.dumps(person, sort_keys=True) for person in persons}
unique_persons_list = [json.loads(p) for p in unique_persons]
        
json_obj = {"persons": unique_persons_list}
print(json.dumps(json_obj, indent=2, ensure_ascii=False))

In [ ]:
with open(f"data/json/{image_name}.json", "w", encoding='utf-8') as json_file:
    json.dump(json_obj, json_file, ensure_ascii=False, indent=2)#!/usr/bin/env python3

### Calculate IE Precison * Recall

In [ ]:
from src.metrics import best_match_similarity
# from src.metrics import calculate_normalized_information_distance

with open(f"data/json/{image_name}.json", 'r', encoding='utf-8') as f:
    constructed_html = json.load(f)
with open(os.path.join("data/labels/info", image_name.replace('.jpg', '.json')), 'r', encoding='utf-8') as f:
    label_html = json.load(f)

# calculate_normalized_information_distance(constructed_html, label_html)
print(best_match_similarity(label_html.get("persons", []), constructed_html.get("persons", [])))


In [ ]:
import json , os
from src.metrics import infomration_extraction_precision_recall

image_name = "NL-HaNA_2.10.50_45_0091.jpg"

with open(f"data/json/{image_name}.json", 'r', encoding='utf-8') as f:
    constructed_html = json.load(f)
with open(os.path.join("data/labels/info", image_name.replace('.jpg', '.json')), 'r', encoding='utf-8') as f:
    label_html = json.load(f)

# calculate_normalized_information_distance(constructed_html, label_html)
print(infomration_extraction_precision_recall(label_html.get("persons", []), constructed_html.get("persons", []), threshold=0.4)) # distance threshold


# KG Construction

In [ ]:
# load json from file
with open(f"data/json/{image_name}.json", "r", encoding="utf-8") as f:
    json_obj = json.load(f)


In [ ]:
# CONSTRUCT ASSERSION TRIPLES
from rdflib import Graph, ConjunctiveGraph, Namespace, URIRef, Literal, RDF

FOAF = Namespace("http://xmlns.com/foaf/0.1/")
EX = Namespace("http://example.org/ontology/")
PROV = Namespace("http://www.w3.org/ns/prov#")

cg = ConjunctiveGraph()
cg.bind("foaf", FOAF)
cg.bind("ex", EX)
cg.bind("prov", PROV)

# Mapping from json keys to RDF predicates
predicate_map = {
    "vader": EX.vader,
    "moeder": EX.moeder,
    "geboorte_datum": EX.geboorteDatum,
    "geboorte_plaats": EX.geboortePlaats,
    "laatste_woonplaats": EX.laatsteWoonplaats
}

for idx, person in enumerate(json_obj["persons"], start=1):
    person_uri = URIRef(f"http://example.org/person/{idx}")
    assertion_graph_uri = URIRef("http://example.org/assertion")
    assertion_graph = Graph(store=cg.store, identifier=assertion_graph_uri)
    assertion_graph.add((person_uri, RDF.type, FOAF.Person))

    provenance_graph_uri = URIRef("http://example.org/provenance")
    provenance_graph = Graph(store=cg.store, identifier=provenance_graph_uri)

    for key, value_dict in person.items():
        value = value_dict["value"]
        cell_id = value_dict["cell"]
        predicate = predicate_map.get(key)
        if predicate:
            # Named graph for each cell
            graph_uri = URIRef(f"http://example.org/graph/{cell_id}")
            ng = Graph(store=cg.store, identifier=graph_uri)
            ng.add((person_uri, predicate, Literal(value)))
            

cg.serialize(f"data/triples/{image_name}_assersion.trig", format='trig')


In [ ]:
end_time = "2025-09-01T12:00:00Z"  # Example end time, replace with actual time if needed
start_time = "2025-09-01T10:00:00Z"  # Example start time, replace with actual time if needed

# CONSTRUCT PROVENANCE TRIPLES
from lxml import etree

def add_provenance_graph(pagexml_path, stamboek_nummer=image_name):
    tree = etree.parse(pagexml_path)
    root = tree.getroot()

    EX = Namespace("http://example.org/ontology/")
    IMG = Namespace("http://example.org/image_ontology/")
    RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")  
    RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
    PROV = Namespace("http://www.w3.org/ns/prov#")
    CSVW = Namespace("http://www.w3.org/ns/csvw#")

    # Create RDF graph
    g = Graph()
    g.bind("ex", EX)
    g.bind("img", IMG)
    g.bind("rdf", RDF)
    g.bind("rdfs", RDFS)
    g.bind("prov", PROV)
    g.csvw = ("csvw", CSVW)


    # Find TableRegion(s)
    table_regions = root.findall(".//{*}TableRegion")

    for table_region in table_regions:
        for cell in table_region.findall(".//{*}TableCell"):
            cell_id = cell.get('id')
            rows = cell.get('row')
            cols = cell.get('col')
            Coords = cell.find(".//{*}Coords")
            coords_points = Coords.get('points') if Coords is not None else None
            
            # cell uri
            named_graph_uri = URIRef(f"http://example.org/graph/{cell_id}")
            cell_uri = URIRef(f"http://example.org/id/{cell_id}")
            provenance_graph.add((named_graph_uri, PROV.wasDerivedFrom, cell_uri))
            g.add((cell_uri, RDF.type, PROV.Entity))
            g.add((cell_uri, RDFS.label, Literal(f"Cell {cell_id} from {stamboek_nummer}")))
            
            g.add((cell_uri, RDF.type, CSVW.Cell))
            g.add((cell_uri, CSVW.rowNumber, Literal(rows)))
            g.add((cell_uri, CSVW.columnNumber, Literal(cols)))
            g.add((cell_uri, EX.ImageRegion, Literal(coords_points)))

            # agents
            agent_1 = URIRef("http://example.org/agent/1")
            g.add((agent_1, RDF.type, PROV.Agent))
            g.add((agent_1, RDFS.label, Literal("Sarah Shoilee")))
            g.add((named_graph_uri, PROV.wasAttributedTo, agent_1))
            project_agent = URIRef("http://example.org/agent/2")
            g.add((project_agent, RDF.type, PROV.Agent))
            g.add((project_agent, RDFS.label, Literal("Pressing Matter Project")))
            g.add((agent_1, PROV.actedOnBehalfOf, project_agent))

            # activity
            stamboekenKGConstructionactivity = URIRef(f"http://example.org/activity/stamboekenKGConstructionactivity/{cell_id}")
            tableConstructionactivity = URIRef(f"http://example.org/activity/TableExtraction/{cell_id}")
            informationExtractionactivity = URIRef(f"http://example.org/activity/InformationExtraction/{cell_id}")
            KGConstructionactivity = URIRef(f"http://example.org/activity/KGConstruction/{cell_id}")
            
            g.add((stamboekenKGConstructionactivity, RDF.type, PROV.Activity))
            g.add((named_graph_uri, PROV.wasGeneratedBy, stamboekenKGConstructionactivity))
            g.add((stamboekenKGConstructionactivity, PROV.wasAssociatedWith, agent_1))
            g.add((stamboekenKGConstructionactivity, PROV.wasInformedBy, tableConstructionactivity))
            g.add((tableConstructionactivity, RDF.type, PROV.Activity))
            g.add((stamboekenKGConstructionactivity, PROV.wasInformedBy, informationExtractionactivity))
            g.add((informationExtractionactivity, RDF.type, PROV.Activity))
            g.add((informationExtractionactivity, PROV.used, cell_uri))
            g.add((stamboekenKGConstructionactivity, PROV.wasInformedBy, KGConstructionactivity))
            g.add((KGConstructionactivity, RDF.type, PROV.Activity))
            g.add((KGConstructionactivity, PROV.used, cell_uri))

            g.add((stamboekenKGConstructionactivity,PROV.endedAtTime, Literal(end_time)))
            g.add((stamboekenKGConstructionactivity,PROV.startedAtTime, Literal(start_time)))

            # Create a Table instance URI
            table_uri = URIRef(f"http://example.org/Table/{cell_id}")
            g.add((table_uri, RDF.type, PROV.Entity))
            g.add((table_uri, RDF.type, CSVW.Table))
            g.add((table_uri, PROV.wasGeneratedBy, tableConstructionactivity))
            g.add((cell_uri, PROV.wasDerivedFrom, table_uri))
            
            # stamboeken
            stamboek_uri = URIRef(f"http://example.org/stamboek/{stamboek_nummer}")
            g.add((stamboek_uri, RDF.type, PROV.Entity))
            g.add((tableConstructionactivity, PROV.used, stamboek_uri))
            g.add((table_uri, PROV.wasDerivedFrom, stamboek_uri))
            national_archives = URIRef("http://example.org/agent/3")
            g.add((national_archives, RDF.type, PROV.Agent))
            g.add((national_archives, RDFS.label, Literal("Nationaal Archief")))
            g.add((stamboek_uri, PROV.wasAttributedTo, national_archives))
    
    g.serialize(f"data/triples/{image_name}_provenance.ttl", format='ttl')
add_provenance_graph(f"data/labels/{image_name}.xml")